In [13]:
import datetime
import os
import random
import sys
import time
sys.path.append(os.path.abspath("../.."))

import numpy as np
import tensorflow as tf
from PIL import Image

from molanet.models.cgan_pix2pix import Pix2PixModel
from molanet.models.cgan_pix2pix import IMAGE_SIZE

## Image Handling

In [11]:
def load_image(name: str, source_dir, target_dir, size=IMAGE_SIZE):
    def transformImageNameSource(name):
        return os.path.join(source_dir, name)

    def transformImageNameTarget(name: str):
        name = name.replace('.jpg', '_Segmentation.png')
        return os.path.join(target_dir, name)

    source_image = Image.open(transformImageNameSource(name))
    target_image = Image.open(transformImageNameTarget(name))

    # TODO think about proper resizing... is dis hacky? I don't know
    size = size, size
    source = source_image.resize(size, Image.BICUBIC)
    target = target_image.resize(size, Image.NEAREST)
    target = target.convert('1')  # to black and white

    return np.array(source).astype(np.float32), np.array(target).astype(np.float32)


def get_image_batch(batch_size, source_file_names, source_dir, target_dir) -> [np.ndarray, np.ndarray]:
    # TODO chances are we don't get fucked by rng
    indices = [random.randint(0, len(source_file_names) - 1) for _ in range(batch_size)]
    images = [load_image(source_file_names[i], source_dir, target_dir) for i in indices]
    return images

def save_ndarrays_asimage(filename: str, *arrays: np.ndarray):
    def fix_dimensions(array):
        if array.ndim > 3 or array.ndim < 2: raise ValueError('arrays must have 2 or 3 dimensions')
        if array.ndim == 2:
            array = np.repeat(array[:, :, np.newaxis], 3, axis=2)  # go from blackwhite to rgb to make concat work seamless
        return array

    if len(arrays) > 1:
        arrays = [fix_dimensions(array) for array in arrays]
        arrays = np.concatenate(arrays, axis=1)

    # arrays is just a big 3-dim matrix
    im = Image.fromarray(np.uint8(arrays))
    im.save(filename)


## Helpers

In [14]:
def save(sess, saver, checkpoint_dir, step):
    model_name = "cgan_pix2pix.model"

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    saver.save(sess, os.path.join(checkpoint_dir, model_name), global_step=step)


def sample_model(filenames: [str], epoch: int, sess: tf.Session, source_dir: str, target_dir: str, model: Pix2PixModel,
                 sample_dir: str, rng: bool):
    batch = None
    if rng:
        batch = get_image_batch(1, filenames, source_dir, target_dir)
    else:
        batch = [load_image('ISIC_0000000.jpg', source_dir, target_dir)]

    (batch_src, batch_target) = batch[0]
    original_source = batch_src.copy()
    original_target = batch_target.copy()
    batch_src = (batch_src / 255.0 - 0.5) * 2.0  # Transform into range -1, 1
    batch_target = (batch_target - 0.5) * 2.0  # Transform into range -1, 1
    batch_src = np.array(batch_src).astype(np.float32)[None, :, :, :]
    batch_target = np.array(batch_target).astype(np.float32)[None, :, :, None]

    sample, d_loss, g_loss = sess.run(
        [model.fake_B, model.d_loss, model.g_loss],
        feed_dict={model.real_data_source: batch_src,
                   model.real_data_target: batch_target}
    )

    # convert images from [-1,1] to [0,255]
    # from shape [1,255,255,1] : tensor to shape [255,255] : nddarray
    sample = (tf.squeeze(sample).eval() + 1.0) / 2.0 * 255

    save_ndarrays_asimage(os.path.join(sample_dir, 'sample_%d.png' % epoch), original_source, sample,
                          original_target * 255)
    print("[Sample] d_loss: {:.8f}, g_loss: {:.8f}".format(d_loss, g_loss))


## Training

In [15]:
source_dir = r'../../../pix2pix-poc-data/training/source'
target_dir = r'../../../pix2pix-poc-data/training/target'
sourcefiles = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

now = datetime.datetime.now()
sample_dir = "./samples/sample-%d-%d-%d--%02d%02d" % (
    now.day, now.month, now.year, now.hour, now.minute)  # Generated samples
checkpoint_dir = "./checkpoints"  # Model
if not os.path.exists('./samples'):
    os.mkdir('./samples')
if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)

use_random_image_as_sample = False
batch_size = 1
size = IMAGE_SIZE
num_feature_maps = 64
is_grayscale = False

restore_iteration = None
iterations = 50000

with tf.Session() as sess:
    model = Pix2PixModel(batch_size=batch_size, image_size=size, src_color_channels=3, target_color_channels=1)

    saver = tf.train.Saver()
    if restore_iteration is not None and restore_iteration > 0:
        iteration_start = restore_iteration + 1
        checkpoint = tf.train.get_checkpoint_state(checkpoint_dir)
        checkpoint_name = os.path.basename(checkpoint.model_checkpoint_path)
        print('checkpoint_name=' + str(checkpoint_name))
        saver.restore(sess, os.path.join(checkpoint_dir, checkpoint_name))
    else:
        iteration_start = 0

    # Optimizers
    disc_optim = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5)  # TODO: pix2pix params
    gen_optim = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5)  # TODO: pix2pix params

    disc_update_step = disc_optim.minimize(model.d_loss, var_list=model.d_vars)
    gen_update_step = gen_optim.minimize(model.g_loss, var_list=model.g_vars)

    sess.run(tf.global_variables_initializer())

    # logging
    writer = tf.summary.FileWriter("./logs", sess.graph)
    g_sum = tf.summary.merge([model.d__sum,
                              model.fake_B_sum, model.d_loss_fake_sum, model.g_loss_sum])
    d_sum = tf.summary.merge([model.d_sum, model.d_loss_real_sum, model.d_loss_sum])

    start_time = time.time()
    for iteration in range(iteration_start, iterations):
        batch = get_image_batch(batch_size, sourcefiles,
                                source_dir=source_dir,
                                target_dir=target_dir)
        (batch_src, batch_target) = batch[0]
        batch_src = (batch_src / 255.0 - 0.5) * 2.0  # Transform into range -1, 1
        batch_target = (batch_target - 0.5) * 2.0  # Transform into range -1, 1

        batch_src = np.array(batch_src).astype(np.float32)[None, :, :, :]
        batch_target = np.array(batch_target).astype(np.float32)[None, :, :, None]

        _, summary_str = sess.run([disc_update_step, d_sum],
                                  feed_dict={model.real_data_source: batch_src,
                                             model.real_data_target: batch_target})

        writer.add_summary(summary_str, iteration)

        # Update G network twice
        _, summary_str = sess.run([gen_update_step, g_sum],
                                  feed_dict={model.real_data_source: batch_src,
                                             model.real_data_target: batch_target})
        writer.add_summary(summary_str, iteration)
        _, summary_str = sess.run([gen_update_step, g_sum],
                                  feed_dict={model.real_data_source: batch_src,
                                             model.real_data_target: batch_target})
        writer.add_summary(summary_str, iteration)

        errD_fake = model.d_loss_fake.eval(
            {model.real_data_target: batch_target, model.real_data_source: batch_src})
        errD_real = model.d_loss_real.eval(
            {model.real_data_target: batch_target, model.real_data_source: batch_src})
        errG = model.g_loss.eval({model.real_data_target: batch_target, model.real_data_source: batch_src})

        print("Epoch: [%2d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
              % (iteration, time.time() - start_time, errD_fake + errD_real, errG))

        if iteration % 50 == 1:
            # gib nice picture output :)
            sample_model(sourcefiles, iteration, sess, source_dir, target_dir, model, sample_dir,
                         use_random_image_as_sample)

        if iteration % 500 == 2:
            save(sess, saver, checkpoint_dir, iteration)



Epoch: [ 0] time: 2.6624, d_loss: 1.27340591, g_loss: 80.57321930
Epoch: [ 1] time: 4.4910, d_loss: 1.24476981, g_loss: 68.81803894
[Sample] d_loss: 1.34016788, g_loss: 83.08406067
Epoch: [ 2] time: 6.7096, d_loss: 1.23761463, g_loss: 75.75146484
Epoch: [ 3] time: 13.8732, d_loss: 1.19189429, g_loss: 55.66500854
Epoch: [ 4] time: 15.2937, d_loss: 1.23487878, g_loss: 110.95969391
Epoch: [ 5] time: 16.7402, d_loss: 1.18900681, g_loss: 53.89413834
Epoch: [ 6] time: 18.1933, d_loss: 1.15724802, g_loss: 44.24517059
Epoch: [ 7] time: 19.6368, d_loss: 1.00687110, g_loss: 48.12762070
Epoch: [ 8] time: 21.1154, d_loss: 1.18261313, g_loss: 36.72450638
Epoch: [ 9] time: 22.5744, d_loss: 1.20217788, g_loss: 36.25959778
Epoch: [10] time: 24.1660, d_loss: 1.15379691, g_loss: 37.27838135
Epoch: [11] time: 26.2300, d_loss: 1.09702504, g_loss: 47.51559830
Epoch: [12] time: 27.7366, d_loss: 1.21805453, g_loss: 24.23547554
Epoch: [13] time: 29.1941, d_loss: 1.04359984, g_loss: 38.02343369
Epoch: [14] tim

KeyboardInterrupt: 